<a href="https://colab.research.google.com/github/byui-cse/cse380-notebooks/blob/master/09_3_About_Huffman_Tree_Growing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About Huffman Tree Growing
## Class Directed Learning
### Due: Tuesday, 2 March 2021, 11:59 pm

## Answer to DPC

This dictionary maps n:cr for the first n odd primes gap lists' cr (compression ratios):


In [1]:
ncr = {2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 25, 8: 28, 9: 25, 10: 23, 11: 25, 12: 23, 13: 21, 14: 20, 15: 22, 16: 21, 17: 19, 18: 21, 19: 20, 20: 19, 21: 18, 22: 0, 23: 0, 24: 2, 25: 2, 26: 4, 27: 4, 28: 32, 29: 33, 30: 32, 31: 32}
for n in ncr.keys():
  print(f"For the first {n} odd primes' gap list, the compression ratio is {ncr[n]}%.")

For the first 2 odd primes' gap list, the compression ratio is 0%.
For the first 3 odd primes' gap list, the compression ratio is 0%.
For the first 4 odd primes' gap list, the compression ratio is 0%.
For the first 5 odd primes' gap list, the compression ratio is 0%.
For the first 6 odd primes' gap list, the compression ratio is 0%.
For the first 7 odd primes' gap list, the compression ratio is 25%.
For the first 8 odd primes' gap list, the compression ratio is 28%.
For the first 9 odd primes' gap list, the compression ratio is 25%.
For the first 10 odd primes' gap list, the compression ratio is 23%.
For the first 11 odd primes' gap list, the compression ratio is 25%.
For the first 12 odd primes' gap list, the compression ratio is 23%.
For the first 13 odd primes' gap list, the compression ratio is 21%.
For the first 14 odd primes' gap list, the compression ratio is 20%.
For the first 15 odd primes' gap list, the compression ratio is 22%.
For the first 16 odd primes' gap list, the comp

### Calculated how??

In [2]:
!pip install anytree
from anytree import Node, RenderTree, PreOrderIter, Walker
from anytree.util import leftsibling, rightsibling
from anytree.exporter.dotexporter import DotExporter
from collections import Counter
from queue import PriorityQueue
from math import ceil, floor, log
from sympy import primerange

def realprimes_up_to(n):
  return list(primerange(4, n))

def get_list_of_gaps(pl):
  gaps_list = list(map(lambda i: pl[i] - pl[i - 1], range(1, len(pl))))
  gaps_list = [2] + gaps_list # [2] for the gap between 3 and 5
  return gaps_list

class GapNode(Node):
  def __lt__(self, other):
    return self.count < other.count

node_counter = 0
def next_node_name():
  global node_counter
  name = 'gn' + str(node_counter)
  node_counter += 1
  return name

def new_node(gp, ct):
  return GapNode(next_node_name(), gap = gp, count = ct)

def new_internal_node(left, right):
  return GapNode(next_node_name(), children = [left, right],
                 gap = 0, count = left.count + right.count)

def make_huffman_tree(gaps_list):
  gap_dict = Counter(gaps_list)
  q = PriorityQueue()
  for key, val in gap_dict.items():
    q.put(new_node(key, val))

  while q.qsize() > 1:
    left = q.get()
    right = q.get()
    q.put(new_internal_node(left, right))

  return q.get()

def get_codes(root):
  leaves = [node for node in PreOrderIter(root, filter_=lambda n: not n.children)]
  codes = {}
  w = Walker()
  for leaf in leaves:
    path = w.walk(leaf, root)[0]
    code = ''
    for node in path:
      code = ('1' if leftsibling(node) else '0') + code
    codes[leaf.gap] = tuple([code, leaf.count])
  return codes

def compression_ratio(f, v):
  return 100 * (f - v) / f

def get_encoded_size(codes):
  return sum([len(code) * count for gap, (code, count) in codes.items()])

def get_fixed_size(codes):
  num_keys = len(codes)
  num_bits_per_key = ceil(log(num_keys, 2))
  return sum([num_bits_per_key * count for gap, (code, count) in codes.items()])

def report(codes):
  return compression_ratio(get_fixed_size(codes), get_encoded_size(codes))

def test_up_to(primes, upper, results):
  list_of_gaps = get_list_of_gaps(primes[:upper])
  print(primes[:upper])
  print(list_of_gaps)
  root = make_huffman_tree(list_of_gaps)
  print(RenderTree(root))
  DotExporter(root).to_picture(f'gap-tree-{upper:02d}.png')
  codes = get_codes(root)
  print(upper, '-->', get_encoded_size(codes))
  cr = round(report(codes))
  results[upper - 1] = cr # adjust since upper is the number of primes, -1 to make it just odd primes

primes = realprimes_up_to(10001)
results = {}
for n in range(3, 33):
    test_up_to(primes, n, results)

results

You should consider upgrading via the 'c:\users\aaron\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
[5, 7, 11]
[2, 2, 4]
GapNode('/gn2', count=3, gap=0)
├── GapNode('/gn2/gn1', count=1, gap=4)
└── GapNode('/gn2/gn0', count=2, gap=2)


FileNotFoundError: [WinError 2] The system cannot find the file specified

## Explore and Wonder

The Huffman Tree Algorithm is one of my favorite algorithms!

### Reason #1

It is easy to understand and implement --- a classic greedy algorithm.

### Reason #2

It is provably optimal among methods encoding symbols separately.

### Reason #3

I think David Huffman was inspired!

#### History

From the History section of Wikipedia's page on Huffman coding:

> In 1951, David A. Huffman and his MIT information theory classmates were given the choice of a term paper or a final exam. The professor, Robert M. Fano, assigned a term paper on the problem of finding the most efficient binary code. Huffman, unable to prove any codes were the most efficient, was about to give up and start studying for the final when he hit upon the idea of using a frequency-sorted binary tree and quickly proved this method the most efficient.

> In doing so, Huffman outdid Fano, who had worked with information theory inventor Claude Shannon to develop a similar code. Building the tree from the bottom up guaranteed optimality, unlike top-down Shannon-Fano coding.





## TODO Read and Draw

Read [A Method for the Construction of Minimum-Redundancy Codes](http://compression.ru/download/articles/huff/huffman_1952_minimum-redundancy-codes.pdf) (David Huffman's original paper) and by drawing pictures, make the connection between trees and rivers vivid in your mind!

In [ ]:
# Trees and rivers

# Trees are represented as one node branching into many children, like so:

'''
         1
      /    \
     2      3
    / \    / \
   4   5  6   7
'''

# Huffman describes a river as a series of brooks and creeks into a larger river, much like an inverse tree:
'''
4   5  6   7
 \  /   \ /
   2     3
    \   /
      1
'''

# These are essentially the same graph! In fact, when we create a Huffman tree "from scratch", we start with the small 'brooks and creeks' of the individual characters and their frequencies. Those rivulets gradually combine, until they form one single large 'river' of code. Take, for example, the following queue that we used in the DPC, which looks visually like a kind of river (many flowing into one):
'''
b1 e1 f1 h1 k1 n1 r1 t1 m2 o5
f1 h1 k1 n1 r1 t1 m2 be2 o5
k1 n1 r1 t1 m2 be2 fh2 o5
r1 t1 m2 be2 fh2 kn2 o5
m2 be2 fh2 kn2 rt2 o5
fh2 kn2 rt2 mbe4 o5
rt2 meb4 fhkn4 o5
fhkn4 o5 rtmeb6
rtmbe6 fhkno9
rtmbefhkno15
'''

# From this river, we formed a tree (one branching into many):
'''
       rtmbefhkno15
         /        \
     rtmbe6      fhkno9
     /   \        /    \
  rt2   mbe4   fhkn4   o5
  /\    / \     /   \
r1 t1 m2 be2  fh2   kn2
         / \  / \   / \
       b1 e1 f1 h1 k1 n1
'''

# Huffman's point with all these rivers, trees, nodes, and graphs, is that they are a way of navigation. We use the Huffman tree to navigate from the root to a leaf, and at each branch, assign a code digit. 

In [6]:
# With Daniel Strickland's help:

n = 1001
primeList = realprimes_up_to(n)
gapList = get_list_of_gaps(primeList)
reconstructPrime = []
reconstructPrime.append(5)
for x in range(0, n):
    reconstructPrime.append(reconstructPrime[x-1] + gapList[x-1])

for y in range(0, n):
    # comparison

IndexError: list index out of range